In [140]:
import utilities
import pandas as pd
import os

If it's the first time we run the code, we'll download the data from bigquery

In [148]:
dataFileName = "data.json"
if (not os.path.exists(dataFileName)):
    data = utilities.retrieveBigQueryData("bigquery-public-data:google_analytics_sample.ga_sessions_20170801")
    data.to_json(dataFileName, orient="records")

We import the data

In [149]:
data = pd.read_json(dataFileName)

We have 2556 rows of Analytics data, with 16 columns

In [150]:
data.shape

(2556, 16)

Many of the columns are objects that contain nested data. Two columns (customDimensions, hits) are arrays of data.

In [151]:
data.head(3)

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,NaN,1,1501583974,1501583974,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Car...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.248282e+18,NaN,NaN,Organic Search,Not Socially Engaged
1,NaN,1,1501616585,1501616585,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",8.647436e+18,NaN,NaN,Organic Search,Not Socially Engaged
2,NaN,1,1501583344,1501583344,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2.055840e+18,NaN,NaN,Organic Search,Not Socially Engaged


Some of the columns (visitorId, userId, clientId) contain no data (due to privacy matters, with this being a sample dataset).
However, we have to keep in mind that the columns with objects or arrays may contain empty objects/arrays.

In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   visitorId             0 non-null      float64
 1   visitNumber           2556 non-null   int64  
 2   visitId               2556 non-null   int64  
 3   visitStartTime        2556 non-null   int64  
 4   date                  2556 non-null   object 
 5   totals                2556 non-null   object 
 6   trafficSource         2556 non-null   object 
 7   device                2556 non-null   object 
 8   geoNetwork            2556 non-null   object 
 9   customDimensions      2556 non-null   object 
 10  hits                  2556 non-null   object 
 11  fullVisitorId         2556 non-null   object 
 12  userId                0 non-null      object 
 13  clientId              0 non-null      object 
 14  channelGrouping       2556 non-null   object 
 15  socialEngagementType 

Let's print some descriptive statistics of the data

In [123]:
data.describe(include="all")

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
count,0.0,2556.000000,2.556000e+03,2.556000e+03,2556,2556,2556,2556,2556,2556,2556,2556,0,0,2556,2556
unique,NaN,NaN,NaN,NaN,1,1083,177,39,830,351,2556,2293,0,0,7,1
top,NaN,NaN,NaN,NaN,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2169837842795965197,NaN,NaN,Organic Search,Not Socially Engaged
freq,NaN,NaN,NaN,NaN,2556,979,1137,665,223,1372,1,4,NaN,NaN,1346,2556
mean,NaN,2.661189,1.501614e+09,1.501614e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,12.022200,2.280455e+04,2.280423e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000,1.501570e+09,1.501571e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.000000,1.501597e+09,1.501597e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.000000,1.501614e+09,1.501614e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,2.000000,1.501632e+09,1.501632e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


If we try to check for duplicates we get an error, due to the impossibility to confront the object columns data. 

In [105]:
data.duplicated().sum()

TypeError: unhashable type: 'dict'